## Applied Data Science - Capstone Project 
### Week 3 - Part 1
### Segmenting and Clustering Neighborhoods in Toronto

* ##### Explore and cluster the neighborhoods in Toronto.
* ##### Generate maps to visualize your neighborhoods and how they cluster together

##### Import  all libraries that are needed

In [9]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import requests

# convert an address into latitude and longitude values
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

# map rendering library
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

##### Upload the dataset where we saved it as csv file in part 2

In [2]:
df = pd.read_csv('Capstone_part2.csv')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Explore Neighborhoods in Toronto

In [3]:
df_Toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop = True)
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [4]:
df_Toronto.shape

(39, 5)

In [25]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df_Toronto['Latitude'],df_Toronto['Longitude'],df_Toronto['Borough'],df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

#### Clustering & mapping

In [42]:
k=5
toronto_clustering = df_Toronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
#df_Toronto.insert(0, 'Cluster Labels', kmeans.labels_)
df_Toronto.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [40]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood'], df_Toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check The Map.JPG To View The Map in repository